In [ ]:
# Imports
import os
import base64
from datetime import date
import requests
#from abc import ABC
import pandas as pd
from pandas.io.json import json_normalize
import geopandas as gpd

from awhere_classes import AWhereAPI


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Define aWhere API key and secret
api_key = os.environ.get('AWHERE_API_KEY')
api_secret = os.environ.get('AWHERE_API_SECRET')

## Level 2: Subclass - Agronomics

In [ ]:
class Agronomics(AWhereAPI):
    def __init__(self, api_key, api_secret, base_64_encoded_secret_key=None, auth_token=None, api_url=None):
        super(Agronomics, self).__init__(api_key, api_secret,
                                      base_64_encoded_secret_key, auth_token)

        self.api_url = 'https://api.awhere.com/v2/agronomics'

    def get_data():
        pass
        
    @staticmethod
    def extract_data():
        pass

    @staticmethod
    def clean_data(df, lon_lat_cols, drop_cols, name_map):
        """Converts dataframe to geodataframe,
        drops unnecessary columns, and renames
        columns.

        Parameters
        ----------
        df : dataframe
            Input dataframe.

        lon_lat_cols : list
            List containing the column name for longitude (list[0])
            and latitude (list[1]) attributes.

        drop_cols : list (of str)
            List of column names to be dropped.

        name_map : dict
            Dictionaty mapping old columns names (keys)
            to new column names (values).

        Returns
        -------
        gdf : geodataframe
            Cleaned geodataframe.

        Example
        -------
        """
        # Define CRS (EPSG 4326) - make this a parameter?
        crs = {'init': 'epsg:4326'}

        # Rename index - possibly as option, or take care of index prior?
        #df.index.rename('date_rename', inplace=True)

        # Create copy of input dataframe; prevents altering the original
        df_copy = df.copy()

        # Convert to geodataframe
        gdf = gpd.GeoDataFrame(
            df_copy, crs=crs, geometry=gpd.points_from_xy(
                df[lon_lat_cols[0]],
                df[lon_lat_cols[1]])
        )

        # Add lat/lon columns to drop columns list
        drop_cols += lon_lat_cols

        # Drop columns
        gdf.drop(columns=drop_cols, axis=1, inplace=True)

        # Rename columns
        gdf.rename(columns=name_map, inplace=True)

        # Return cleaned up geodataframe
        return gdf
    
#     @classmethod
#     def api_to_gdf(): # (cls, api_object, kwargs=None)
#         pass
    
    @classmethod ## POSSIBLY DEFINE THIS AT CURRENT LEVEL IF STRUCTURES MATCH
    def api_to_gdf(cls, api_object, kwargs=None):
        """kwargs is a dictionary that provides values beyond the default;
        unpack dictionary if it exists
        
        kwargs are the parameters to get_data() method

        kwargs={'start_day': '03-04', 'end_day': '03-07', 'offset': 2}
        """
        api_data_json = api_object.get_data(
            **kwargs) if kwargs else api_object.get_data()

        api_data_df =  cls.extract_data(api_data_json)

        api_data_gdf = cls.clean_data(
            api_data_df,
            cls.coord_cols,
            cls.drop_cols,
            cls.rename_map
        )

        return api_data_gdf

In [ ]:
agro = Agronomics(api_key, api_secret)

In [ ]:
agro.api_url

## Level 3: Sub-sub-class - AgronomicsLocation

In [ ]:
class AgronomicsLocation(Agronomics):
    
    def __init__(self, api_key, api_secret, base_64_encoded_secret_key=None,
                 auth_token=None, api_url=None):

        super(AgronomicsLocation, self).__init__(
            api_key, api_secret, base_64_encoded_secret_key, auth_token)

        self.api_url = f"{self.api_url}/locations"

In [ ]:
agro_locations = AgronomicsLocation(api_key, api_secret)

In [ ]:
agro_locations.api_url

## Level 3: Sub-sub-class - AgronomicsField

In [ ]:
class AgronomicsField(Agronomics):
    
    def __init__(self, api_key, api_secret, base_64_encoded_secret_key=None,
                 auth_token=None, api_url=None):

        super(AgronomicsField, self).__init__(
            api_key, api_secret, base_64_encoded_secret_key, auth_token)

        self.api_url = f"{self.api_url}/fields"

## Level 4: Sub-sub-sub-class - AgronomicsLocationValues

In [ ]:
class AgronomicsLocationValues(AgronomicsLocation):

    # Class variables for clean_data() function
    # Single day
    day_coord_cols = ['location.latitude', 'location.longitude']

    day_drop_cols = ['pet.units', '_links.self.href']

    day_rename_map = {
        "gdd": "gdd_daily_total_cels",
        "ppet": "ppet_daily_total",
        "pet.amount": "pet_daily_total_mm"
    }  
    
    # Multi-day, total accumulation
    total_coord_cols = ['latitude', 'longitude']

    total_drop_cols = ['precipitation.units', 'pet.units']
    
    total_rename_map = { ## PPET overall?? Range total
        "gdd": "gdd_range_total_cels",
        "ppet": "ppet_range_total", # This is accumulation of all daily PPET
        "precipitation.amount": "precip_range_total_mm",
        "pet.amount": "pet_range_total_mm"
    }

    # Multi-day, daily accumulation
    daily_coord_cols = ['latitude', 'longitude']

    daily_drop_cols = ['pet.units', 'accumulatedPrecipitation.units',
                       'accumulatedPet.units', '_links.self.href']

    daily_rename_map = {
        "gdd": "gdd_daily_total_cels",
        "ppet": "ppet_daily_total",
        "accumulatedGdd": "gdd_rolling_total_cels",
        "accumulatedPpet": "ppet_rolling_total",
        "pet.amount": "pet_daily_total_mm",
        "accumulatedPrecipitation.amount": "precip_rolling_total_mm",
        "accumulatedPet.amount": "pet_rolling_total_mm"
    }

    # Define lat/lon when intitializing class; no need to repeat for lat/lon
    #  in get_data() because it is already programmed into api_url
    def __init__(self, api_key, api_secret, latitude, longitude, base_64_encoded_secret_key=None,
                 auth_token=None, api_url=None):

        super(AgronomicsLocationValues, self).__init__(
            api_key, api_secret, base_64_encoded_secret_key, auth_token, api_url)

        self.latitude = latitude
        self.longitude = longitude
        self.api_url = f"{self.api_url}/{self.latitude},{self.longitude}/agronomicvalues"

    def get_data(self, start_day=date.today().strftime("%m-%d"), end_day=None, offset=0):
        """Returns aWhere Forecast Agronomic Values.
        """

        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}"
        }

        # Perform the HTTP request to obtain the norms for the Field
        # Define URL variants
        url_single_day = f"{self.api_url}/{start_day}"
        url_multiple_days = f"{self.api_url}/{start_day},{end_day}?limit=10&offset={offset}"

        # Get single day norms or date range
        response = requests.get(url_multiple_days, headers=auth_headers) if end_day else requests.get(
            url_single_day, headers=auth_headers)

        # Return the norms
        return response.json()

    @staticmethod
    def extract_data(agronomic_values):
        """Extracts data from the aWhere agronomic forecast
        data in JSON format.
        """
        # Extract lat/lon
        latitude = agronomic_values.get('location').get('latitude')
        longitude = agronomic_values.get('location').get('longitude')

        # Check if more than one day
        if agronomic_values.get('dailyValues'):

            # Do these with a separate call, just like in Soil accumulation='daily'
            #  accumulation='total'

            # DAILY ACCUMULATIONS
            # Get daily forecasted accumulations
            daily_accumulation = json_normalize(
                agronomic_values.get('dailyValues'))

            # Add lat/lon and set date as index
            daily_accumulation['latitude'] = latitude
            daily_accumulation['longitude'] = longitude
            daily_accumulation.set_index(['date'], inplace=True)

            # TOTAL ACCUMULATION
            # Get total forecasted accumulations through all days
            total_accumulation = json_normalize(
                agronomic_values.get('accumulations'))

            # Get list of dates, add start/end dates, set date range as index
            dates = [entry.get('date')
                     for entry in agronomic_values.get('dailyValues')]
            total_accumulation['date_range'] = f"{dates[0]}/{dates[-1]}"
            total_accumulation['start_day'] = dates[0]
            total_accumulation['end_day'] = dates[-1]
            total_accumulation.set_index(['date_range'], inplace=True)

            # Add lat/lon
            total_accumulation['latitude'] = latitude
            total_accumulation['longitude'] = longitude

            # Put dataframes in tuple (total accumulation, daily accumulation)
            agronomics_df = (total_accumulation, daily_accumulation)

        # Single day
        else:
            agronomics_df = json_normalize(agronomic_values)
            # agronomics_df['latitude'] = latitude
            # agronomics_df['longitude'] = longitude
            agronomics_df.set_index(['date'], inplace=True)

        return agronomics_df

    @classmethod
    def api_to_gdf(cls, api_object, value_type='single_day', kwargs=None):
        """
        value_type can be 'single_day' or 'multi_day'.

        kwargs is a dictionary that provides values beyond the default;
        unpack dictionary if it exists

        kwargs are the parameters to get_data() method

        kwargs={'start_day': '03-04', 'end_day': '03-07', 'offset': 2}
        """
        api_data_json = api_object.get_data(
            **kwargs) if kwargs else api_object.get_data()

        if value_type.lower() == 'single_day':
            api_data_df = cls.extract_data(api_data_json)

            api_data_gdf = cls.clean_data(
                api_data_df,
                cls.day_coord_cols,
                cls.day_drop_cols,
                cls.day_rename_map
            )

        elif value_type.lower() == 'multi_day':
            api_data_df_total, api_data_df_daily = cls.extract_data(
                api_data_json)

            api_data_gdf_total = cls.clean_data(
                api_data_df_total,
                cls.total_coord_cols,
                cls.total_drop_cols,
                cls.total_rename_map
            )

            api_data_gdf_daily = cls.clean_data(
                api_data_df_daily,
                cls.daily_coord_cols,
                cls.daily_drop_cols,
                cls.daily_rename_map
            )

            api_data_gdf = (api_data_gdf_total, api_data_gdf_daily)

        else:
            raise ValueError("Invalid value type. Please choose 'single_day' or 'multi_day'.")

        return api_data_gdf

## Level 4: Sub-sub-sub-class - AgronomicsLocationNorms

In [ ]:
class AgronomicsLocationNorms(AgronomicsLocation):

    # Class variables for clean_data() function
    
    # https://developer.awhere.com/api/reference/agronomics/norms/geolocation
    
    """The average ratio of Precipitation to Potential Evapotranspiration 
    over the years specified. When this value is above 1, then more rain fell 
    than the amount of likely water loss; if it's below 1, then more water was
    likely lost than fell as rain. P/PET is most useful when calculated for a 
    range of days, as it is for this property, than for individual days."""
    
    # Single day   
    day_coord_cols = ['location.latitude', 'location.longitude']

    day_drop_cols = ['pet.units', '_links.self.href']
     
    day_rename_map = {
        "gdd.average": "gdd_daily_average_total_cels",
        "gdd.stdDev": "gdd_daily_average_total_std_dev_cels",
        "pet.average": "pet_daily_average_total_mm",
        "pet.stdDev": "pet_daily_average_total_std_dev_mm",
        "ppet.average": "ppet_daily_average_total",
        "ppet.stdDev": "ppet_daily_average_total_std_dev"
    }
     
    # Multi-day, total accumulation
    total_coord_cols = ['latitude', 'longitude']

    total_drop_cols = ['precipitation.units', 'pet.units']

    total_rename_map = {
        "gdd.average": "norms_gdd_average_total_cels",
        "gdd.stdDev": "norms_gdd_average_total_std_dev_cels",
        "precipitation.average": "norms_precip_average_total_mm",
        "precipitation.stdDev": "norms_precip_average_total_std_dev_mm",
        "pet.average": "norms_pet_average_total_mm",
        "pet.stdDev": "norms_pet_average_total_std_dev",
        "ppet.average": "norms_ppet_average_total",
        "ppet.stdDev": "norms_ppet_average_total_std_dev"
    }

    total_rename_map = {
        "gdd.average": "gdd_range_average_total_cels",
        "gdd.stdDev": "gdd_range_average_total_std_dev_cels",
        "precipitation.average": "precip_range_average_total_mm",
        "precipitation.stdDev": "precip_range_average_total_std_dev_mm",
        "pet.average": "pet_range_average_total_mm",
        "pet.stdDev": "pet_range_average_total_std_dev",
        # Why doesn't this match with precip_avg/pet_avg? 
        # What causes this difference?
        # Is it the average of each of individual PPET daily values?
        # Seems like it
        "ppet.average": "ppet_range_daily_average", 
        "ppet.stdDev": "ppet_range_daily_average_std_dev"
    }
    
    # Multi-day, daily accumulation
    daily_coord_cols = ['latitude', 'longitude']

    daily_drop_cols = ['pet.units', 'accumulatedPrecipitation.units',
                       'accumulatedPet.units', '_links.self.href']
    
    daily_rename_map = {       
        "gdd.average": "gdd_daily_average_cels",
        "gdd.stdDev": "gdd_daily_average_std_dev_cels",
        "pet.average": "pet_daily_average_mm",
        "pet.stdDev": "pet_daily_average_std_dev_mm",
        "ppet.average": "ppet_daliy_average",
        "ppet.stdDev": "ppet_daily_average_std_dev",
        "accumulatedGdd.average": "gdd_rolling_total_average",
        "accumulatedGdd.stdDev": "gdd_rolling_total_average_std_dev",
        "accumulatedPrecipitation.average": "precip_rolling_total_average_mm",
        "accumulatedPrecipitation.stdDev": "precip_rolling_total_average_std_dev_mm",
        "accumulatedPet.average": "pet_rolling_total_average_mm",
        "accumulatedPet.stdDev": "pet_rolling_total_average_std_dev_mm", 
        "accumulatedPpet.average": "ppet_rolling_total_average",
        "accumulatedPpet.stdDev": "ppet_rolling_total_average_std_dev"
    }

    # Define lat/lon when intitializing class; no need to repeat for lat/lon
    #  in get_data() because it is already programmed into api_url
    def __init__(self, api_key, api_secret, latitude, longitude, base_64_encoded_secret_key=None,
                 auth_token=None, api_url=None):

        super(AgronomicsLocationNorms, self).__init__(
            api_key, api_secret, base_64_encoded_secret_key, auth_token, api_url)

        self.latitude = latitude
        self.longitude = longitude
        self.api_url = f"{self.api_url}/{self.latitude},{self.longitude}/agronomicnorms"

    def get_data(self, start_day='01-01', end_day=None, offset=0):
        """Returns aWhere Historic Agronomic Norms.
        """

        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}"
        }

        # Perform the HTTP request to obtain the norms for the Field
        # Define URL variants
        url_single_day = f"{self.api_url}/{start_day}"
        url_multiple_days = f"{self.api_url}/{start_day},{end_day}?limit=10&offset={offset}"

        # Get single day norms or date range
        response = requests.get(url_multiple_days, headers=auth_headers) if end_day else requests.get(
            url_single_day, headers=auth_headers)

        # Return the norms
        return response.json()

    @staticmethod
    def extract_data(agronomic_norms):
        """Extracts data from the aWhere agronomic norms
        data in JSON format.
        """
        # Extract lat/lon
        latitude = agronomic_norms.get('location').get('latitude')
        longitude = agronomic_norms.get('location').get('longitude')

        # Check if more than one day
        if agronomic_norms.get('dailyNorms'):

            # DAILY ACCUMULATION NORMS
            # Get daily accumulation norms
            daily_norms = json_normalize(
                agronomic_norms.get('dailyNorms'))

            # Add lat/lon and set date as index
            daily_norms['latitude'] = latitude
            daily_norms['longitude'] = longitude
            daily_norms.set_index(['day'], inplace=True)

            # TOTAL ACCUMULATION NORMS
            # Get average accumulations through all days
            total_norms = json_normalize(
                agronomic_norms.get('averageAccumulations'))

            # Get list of dates, add start/end dates, set date range as index
            dates = [entry.get('day')
                     for entry in agronomic_norms.get('dailyNorms')]
            total_norms['date_range'] = f"{dates[0]}/{dates[-1]}"
            total_norms['start_day'] = dates[0]
            total_norms['end_day'] = dates[-1]
            total_norms.set_index(['date_range'], inplace=True)

            # Add lat/lon
            total_norms['latitude'] = latitude
            total_norms['longitude'] = longitude

            # Put dataframes in tuple (total norms, daily norms)
            agronomics_df = (total_norms, daily_norms)

        # Single day
        else:
            agronomics_df = json_normalize(agronomic_norms)
            # agronomics_df['latitude'] = latitude
            # agronomics_df['longitude'] = longitude
            agronomics_df.set_index(['day'], inplace=True)

        return agronomics_df

    @classmethod
    def api_to_gdf(cls, api_object, value_type='single_day', kwargs=None):
        """
        value_type can be 'single_day' or 'multi_day'.

        kwargs is a dictionary that provides values beyond the default;
        unpack dictionary if it exists

        kwargs are the parameters to get_data() method

        kwargs={'start_day': '03-04', 'end_day': '03-07', 'offset': 2}
        """
        api_data_json = api_object.get_data(
            **kwargs) if kwargs else api_object.get_data()

        if value_type.lower() == 'single_day':
            api_data_df = cls.extract_data(api_data_json)

            api_data_gdf = cls.clean_data(
                api_data_df,
                cls.day_coord_cols,
                cls.day_drop_cols,
                cls.day_rename_map
            )

        elif value_type.lower() == 'multi_day':
            api_data_df_total, api_data_df_daily = cls.extract_data(
                api_data_json)

            api_data_gdf_total = cls.clean_data(
                api_data_df_total,
                cls.total_coord_cols,
                cls.total_drop_cols,
                cls.total_rename_map
            )

            api_data_gdf_daily = cls.clean_data(
                api_data_df_daily,
                cls.daily_coord_cols,
                cls.daily_drop_cols,
                cls.daily_rename_map
            )

            api_data_gdf = (api_data_gdf_total, api_data_gdf_daily)

        else:
            raise ValueError("Invalid value type. Please choose 'single_day' or 'multi_day'.")

        return api_data_gdf

## Level 4: Sub-sub-sub-class - AgronomicsFieldValues

In [ ]:
class AgronomicsFieldValues(AgronomicsField):

    # Class variables for clean_data() function
    # Single day
    day_coord_cols = ['location.latitude', 'location.longitude']

    day_drop_cols = ['location.fieldId', 'pet.units', '_links.self.href',
                     '_links.curies', '_links.awhere:field.href']

    day_rename_map = {
        "gdd": "gdd_daily_total_cels",
        "ppet": "ppet_daily_total",
        "pet.amount": "pet_daily_total_mm"
    }

    # Multi-day, total accumulation
    total_coord_cols = ['latitude', 'longitude']

    total_drop_cols = ['precipitation.units', 'pet.units']

    total_rename_map = { ## PPET overall?? Range total
        "gdd": "gdd_range_total_cels",
        "ppet": "ppet_range_total", # This is accumulation of all daily PPET
        "precipitation.amount": "precip_range_total_mm",
        "pet.amount": "pet_range_total_mm"
    }

    # Multi-day, daily accumulation
    daily_coord_cols = ['latitude', 'longitude']

    daily_drop_cols = ['pet.units', 'accumulatedPrecipitation.units',
                       'accumulatedPet.units', '_links.self.href',
                       '_links.curies', '_links.awhere:field.href']

    daily_rename_map = {
        "gdd": "gdd_daily_total_cels",
        "ppet": "ppet_daily_total",
        "accumulatedGdd": "gdd_rolling_total_cels",
        "accumulatedPpet": "ppet_rolling_total",
        "pet.amount": "pet_daily_total_mm",
        "accumulatedPrecipitation.amount": "precip_rolling_total_mm",
        "accumulatedPet.amount": "pet_rolling_total_mm"
    }

    # Define lat/lon when intitializing class; no need to repeat for lat/lon
    #  in get_data() because it is already programmed into api_url
    def __init__(self, api_key, api_secret, field_id, base_64_encoded_secret_key=None,
                 auth_token=None, api_url=None):

        super(AgronomicsFieldValues, self).__init__(
            api_key, api_secret, base_64_encoded_secret_key, auth_token, api_url)

        self.field_id = field_id
        self.api_url = f"{self.api_url}/{self.field_id}/agronomicvalues"

    def get_data(self, start_day=date.today().strftime("%m-%d"), end_day=None, offset=0):
        """Returns aWhere Forecast Agronomic Values for a provided field.
        """

        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}"
        }

        # Perform the HTTP request to obtain the norms for the Field
        # Define URL variants
        url_single_day = f"{self.api_url}/{start_day}"
        url_multiple_days = f"{self.api_url}/{start_day},{end_day}?limit=10&offset={offset}"

        # Get single day norms or date range
        response = requests.get(url_multiple_days, headers=auth_headers) if end_day else requests.get(
            url_single_day, headers=auth_headers)

        # Return the norms
        return response.json()

    @staticmethod
    def extract_data(agronomic_values):
        """Extracts data from the aWhere agronomic forecast
        data in JSON format.
        """
        # Extract lat/lon
        latitude = agronomic_values.get('location').get('latitude')
        longitude = agronomic_values.get('location').get('longitude')

        # Check if more than one day
        if agronomic_values.get('dailyValues'):

            # Do these with a separate call, just like in Soil accumulation='daily'
            #  accumulation='total'

            # DAILY ACCUMULATIONS
            # Get daily forecasted accumulations
            daily_accumulation = json_normalize(
                agronomic_values.get('dailyValues'))

            # Add lat/lon and set date as index
            daily_accumulation['latitude'] = latitude
            daily_accumulation['longitude'] = longitude
            daily_accumulation.set_index(['date'], inplace=True)

            # TOTAL ACCUMULATION
            # Get total forecasted accumulations through all days
            total_accumulation = json_normalize(
                agronomic_values.get('accumulations'))

            # Get list of dates, add start/end dates, set date range as index
            dates = [entry.get('date')
                     for entry in agronomic_values.get('dailyValues')]
            total_accumulation['date_range'] = f"{dates[0]}/{dates[-1]}"
            total_accumulation['start_day'] = dates[0]
            total_accumulation['end_day'] = dates[-1]
            total_accumulation.set_index(['date_range'], inplace=True)

            # Add lat/lon
            total_accumulation['latitude'] = latitude
            total_accumulation['longitude'] = longitude

            # Put dataframes in tuple (total accumulation, daily accumulation)
            agronomics_df = (total_accumulation, daily_accumulation)

        # Single day
        else:
            agronomics_df = json_normalize(agronomic_values)
            # agronomics_df['latitude'] = latitude
            # agronomics_df['longitude'] = longitude
            agronomics_df.set_index(['date'], inplace=True)

        return agronomics_df

    @classmethod
    def api_to_gdf(cls, api_object, value_type='single_day', kwargs=None):
        """
        value_type can be 'single_day' or 'multi_day'.

        kwargs is a dictionary that provides values beyond the default;
        unpack dictionary if it exists

        kwargs are the parameters to get_data() method

        kwargs={'start_day': '03-04', 'end_day': '03-07', 'offset': 2}
        """
        api_data_json = api_object.get_data(
            **kwargs) if kwargs else api_object.get_data()

        if value_type.lower() == 'single_day':
            api_data_df = cls.extract_data(api_data_json)

            api_data_gdf = cls.clean_data(
                api_data_df,
                cls.day_coord_cols,
                cls.day_drop_cols,
                cls.day_rename_map
            )

        elif value_type.lower() == 'multi_day':
            api_data_df_total, api_data_df_daily = cls.extract_data(
                api_data_json)

            api_data_gdf_total = cls.clean_data(
                api_data_df_total,
                cls.total_coord_cols,
                cls.total_drop_cols,
                cls.total_rename_map
            )

            api_data_gdf_daily = cls.clean_data(
                api_data_df_daily,
                cls.daily_coord_cols,
                cls.daily_drop_cols,
                cls.daily_rename_map
            )

            api_data_gdf = (api_data_gdf_total, api_data_gdf_daily)

        else:
            raise ValueError("Invalid value type. Please choose 'single_day' or 'multi_day'.")

        return api_data_gdf

In [ ]:
agro_field = AgronomicsFieldValues(
    api_key, api_secret, field_id='CO-RMNP-Bear-Lake')

In [ ]:
agro_field = AgronomicsFieldValues(
    api_key, api_secret, field_id='CO-RMNP-Bear-Lake')

AgronomicsFieldValues.api_to_gdf(agro_field, 
    value_type='single_day', 
    kwargs=None)

total, daily = AgronomicsFieldValues.api_to_gdf(agro_field, 
    value_type='multi_day', 
    kwargs={'start_day': '03-03', 'end_day': '03-09'})

total

daily

In [ ]:
total, daily = AgronomicsFieldValues.api_to_gdf(agro_field, 
    value_type='multi_day', 
    kwargs={'start_day': '03-03', 'end_day': '03-09'})

In [ ]:
total

In [ ]:
daily

In [ ]:
agro_field.get_data()

In [ ]:
AgronomicsFieldValues.extract_data(agro_field.get_data())

## Level 4: Sub-sub-sub-class - AgronomicsFieldNorms

In [ ]:
class AgronomicsFieldNorms(AgronomicsField):

    # Class variables for clean_data() function
    
    # https://developer.awhere.com/api/reference/agronomics/norms/geolocation
    
    """The average ratio of Precipitation to Potential Evapotranspiration 
    over the years specified. When this value is above 1, then more rain fell 
    than the amount of likely water loss; if it's below 1, then more water was
    likely lost than fell as rain. P/PET is most useful when calculated for a 
    range of days, as it is for this property, than for individual days."""
    
    # Single day   
    day_coord_cols = ['location.latitude', 'location.longitude']

    day_drop_cols = ['location.fieldId', 'pet.units', '_links.self.href',
                     '_links.curies', '_links.awhere:field.href']

    day_rename_map = {
        "gdd.average": "gdd_daily_average_total_cels",
        "gdd.stdDev": "gdd_daily_average_total_std_dev_cels",
        "pet.average": "pet_daily_average_total_mm",
        "pet.stdDev": "pet_daily_average_total_std_dev_mm",
        "ppet.average": "ppet_daily_average_total",
        "ppet.stdDev": "ppet_daily_average_total_std_dev"
    }
     
    # Multi-day, total accumulation
    total_coord_cols = ['latitude', 'longitude']

    total_drop_cols = ['precipitation.units', 'pet.units']

    total_rename_map = {
        "gdd.average": "gdd_range_average_total_cels",
        "gdd.stdDev": "gdd_range_average_total_std_dev_cels",
        "precipitation.average": "precip_range_average_total_mm",
        "precipitation.stdDev": "precip_range_average_total_std_dev_mm",
        "pet.average": "pet_range_average_total_mm",
        "pet.stdDev": "pet_range_average_total_std_dev",
        # Why doesn't this match with precip_avg/pet_avg? 
        # What causes this difference?
        # Is it the average of each of individual PPET daily values?
        # Seems like it
        "ppet.average": "ppet_range_daily_average", 
        "ppet.stdDev": "ppet_range_daily_average_std_dev"
    }

    # Multi-day, daily accumulation
    daily_coord_cols = ['latitude', 'longitude']

    daily_drop_cols = ['pet.units', 'accumulatedPrecipitation.units',
                       'accumulatedPet.units', '_links.self.href',
                       '_links.curies', '_links.awhere:field.href']

    daily_rename_map = {       
        "gdd.average": "gdd_daily_average_cels",
        "gdd.stdDev": "gdd_daily_average_std_dev_cels",
        "pet.average": "pet_daily_average_mm",
        "pet.stdDev": "pet_daily_average_std_dev_mm",
        "ppet.average": "ppet_daliy_average",
        "ppet.stdDev": "ppet_daily_average_std_dev",
        "accumulatedGdd.average": "gdd_rolling_total_average",
        "accumulatedGdd.stdDev": "gdd_rolling_total_average_std_dev",
        "accumulatedPrecipitation.average": "precip_rolling_total_average_mm",
        "accumulatedPrecipitation.stdDev": "precip_rolling_total_average_std_dev_mm",
        "accumulatedPet.average": "pet_rolling_total_average_mm",
        "accumulatedPet.stdDev": "pet_rolling_total_average_std_dev_mm", 
        "accumulatedPpet.average": "ppet_rolling_total_average",
        "accumulatedPpet.stdDev": "ppet_rolling_total_average_std_dev"
    }

    # Define lat/lon when intitializing class; no need to repeat for lat/lon
    #  in get_data() because it is already programmed into api_url
    def __init__(self, api_key, api_secret, field_id, base_64_encoded_secret_key=None,
                 auth_token=None, api_url=None):

        super(AgronomicsFieldNorms, self).__init__(
            api_key, api_secret, base_64_encoded_secret_key, auth_token, api_url)

        self.field_id = field_id
        self.api_url = f"{self.api_url}/{self.field_id}/agronomicnorms"

    def get_data(self, start_day='01-01', end_day=None, offset=0):
        """Returns aWhere Historic Agronomic Norms.
        """

        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}"
        }

        # Perform the HTTP request to obtain the norms for the Field
        # Define URL variants
        url_single_day = f"{self.api_url}/{start_day}"
        url_multiple_days = f"{self.api_url}/{start_day},{end_day}?limit=10&offset={offset}"

        # Get single day norms or date range
        response = requests.get(url_multiple_days, headers=auth_headers) if end_day else requests.get(
            url_single_day, headers=auth_headers)

        # Return the norms
        return response.json()

    @staticmethod
    def extract_data(agronomic_norms):
        """Extracts data from the aWhere agronomic norms
        data in JSON format.
        """
        # Extract lat/lon
        latitude = agronomic_norms.get('location').get('latitude')
        longitude = agronomic_norms.get('location').get('longitude')

        # Check if more than one day
        if agronomic_norms.get('dailyNorms'):

            # DAILY ACCUMULATION NORMS
            # Get daily accumulation norms
            daily_norms = json_normalize(
                agronomic_norms.get('dailyNorms'))

            # Add lat/lon and set date as index
            daily_norms['latitude'] = latitude
            daily_norms['longitude'] = longitude
            daily_norms.set_index(['day'], inplace=True)

            # TOTAL ACCUMULATION NORMS
            # Get average accumulations through all days
            total_norms = json_normalize(
                agronomic_norms.get('averageAccumulations'))

            # Get list of dates, add start/end dates, set date range as index
            dates = [entry.get('day')
                     for entry in agronomic_norms.get('dailyNorms')]
            total_norms['date_range'] = f"{dates[0]}/{dates[-1]}"
            total_norms['start_day'] = dates[0]
            total_norms['end_day'] = dates[-1]
            total_norms.set_index(['date_range'], inplace=True)

            # Add lat/lon
            total_norms['latitude'] = latitude
            total_norms['longitude'] = longitude

            # Put dataframes in tuple (total norms, daily norms)
            agronomics_df = (total_norms, daily_norms)

        # Single day
        else:
            agronomics_df = json_normalize(agronomic_norms)
            # agronomics_df['latitude'] = latitude
            # agronomics_df['longitude'] = longitude
            agronomics_df.set_index(['day'], inplace=True)

        return agronomics_df

    @classmethod
    def api_to_gdf(cls, api_object, value_type='single_day', kwargs=None):
        """
        value_type can be 'single_day' or 'multi_day'.

        kwargs is a dictionary that provides values beyond the default;
        unpack dictionary if it exists

        kwargs are the parameters to get_data() method

        kwargs={'start_day': '03-04', 'end_day': '03-07', 'offset': 2}
        """
        api_data_json = api_object.get_data(
            **kwargs) if kwargs else api_object.get_data()

        if value_type.lower() == 'single_day':
            api_data_df = cls.extract_data(api_data_json)

            api_data_gdf = cls.clean_data(
                api_data_df,
                cls.day_coord_cols,
                cls.day_drop_cols,
                cls.day_rename_map
            )

        elif value_type.lower() == 'multi_day':
            api_data_df_total, api_data_df_daily = cls.extract_data(
                api_data_json)

            api_data_gdf_total = cls.clean_data(
                api_data_df_total,
                cls.total_coord_cols,
                cls.total_drop_cols,
                cls.total_rename_map
            )

            api_data_gdf_daily = cls.clean_data(
                api_data_df_daily,
                cls.daily_coord_cols,
                cls.daily_drop_cols,
                cls.daily_rename_map
            )

            api_data_gdf = (api_data_gdf_total, api_data_gdf_daily)

        else:
            raise ValueError("Invalid value type. Please choose 'single_day' or 'multi_day'.")

        return api_data_gdf

In [ ]:
agro_field = AgronomicsFieldNorms(
    api_key, api_secret, field_id='CO-RMNP-Bear-Lake')

In [ ]:
agro_field.get_data()

In [ ]:
AgronomicsFieldNorms.api_to_gdf(agro_field)

In [ ]:
total, daily = AgronomicsFieldNorms.api_to_gdf(agro_field, 
    value_type='multi_day', 
    kwargs={'start_day': '03-20', 'end_day': '03-25'})

In [ ]:
total

In [ ]:
daily

In [ ]:
# Define AgronomicsLocationValues object
agro = AgronomicsLocationValues(
    api_key, api_secret, latitude=40.313250, longitude=-105.648222)

In [ ]:
# Single day test (default)
agro_json = agro.get_data()
AgronomicsLocationValues.extract_data(agro_json)
AgronomicsLocationValues.clean_data(
    AgronomicsLocationValues.extract_data(agro_json),
    AgronomicsLocationValues.day_coord_cols,
    AgronomicsLocationValues.day_drop_cols,
    AgronomicsLocationValues.day_rename_map)

In [ ]:
# Multiday test - total forecasted accumulations
agro_json = agro.get_data(end_day='04-23')
total_accum, daily_accum = AgronomicsLocationValues.extract_data(agro_json)
AgronomicsLocationValues.clean_data(
    total_accum,
    AgronomicsLocationValues.total_coord_cols,
    AgronomicsLocationValues.total_drop_cols,
    AgronomicsLocationValues.total_rename_map)

In [ ]:
# Multi day test - dialy accumulations
AgronomicsLocationValues.clean_data(
    daily_accum,
    AgronomicsLocationValues.daily_coord_cols,
    AgronomicsLocationValues.daily_drop_cols,
    AgronomicsLocationValues.daily_rename_map)

In [ ]:
# Define AgronomicsLocationValues object - single day
agro = AgronomicsLocationValues(
    api_key, api_secret, latitude=40.313250, longitude=-105.648222)
AgronomicsLocationValues.api_to_gdf(agro)

In [ ]:
# Define AgronomicsLocationValues object - single day
agro = AgronomicsLocationValues(
    api_key, api_secret, latitude=40.313250, longitude=-105.648222)

In [ ]:
total_accum, daily_accum = AgronomicsLocationValues.api_to_gdf(
    agro, value_type='multi_day', kwargs={'end_day': '04-23'})

In [ ]:
total_accum

In [ ]:
daily_accum

In [ ]:
date.today().strftime("%m-%d")

In [ ]:
agro_json = agro.get_data()

In [ ]:
agro_loc_values = AgronomicsLocationValues(
    api_key, api_secret, latitude=40.313250, longitude=-105.648222)

In [ ]:
agro_loc_values.api_url

In [ ]:
agro_loc_values.latitude

In [ ]:
agro_loc_values.longitude

In [ ]:
# Single day
agro_loc_values.get_data()

In [ ]:
# Dictionary keys
agro_loc_values.get_data().keys()

In [ ]:
agro_loc_values.get_data(start_day='04-20')

In [ ]:
# Flattened to df
json_normalize(agro_loc_values.get_data(start_day='04-20'))

In [ ]:
# Range of days
agro_loc_values.get_data(end_day='04-24')

In [ ]:
agro_loc_values.get_data(end_day='04-24').keys()

In [ ]:
multi_day = agro_loc_values.get_data(end_day='04-24')

In [ ]:
# Top level flatten
json_normalize(multi_day)

In [ ]:
# Accumulations forecast data
multi_day.get('accumulations')

In [ ]:
# Accumulations forcast data - flattened
json_normalize(multi_day.get('accumulations'))

In [ ]:
# Daily values forecast
multi_day.get('dailyValues')

# You have daily GDD, PET, and PPET, and 
#  accumulatd GDD, accumulated Precip, accumulated PET, and accumulated PPET 

In [ ]:
# Daily values forecast - flattened
json_normalize(multi_day.get('dailyValues'))

Have clean_data() return two things, or have two functions, like with Soil? extract_daily() vs extract_accumulated()? 

In [ ]:
# Difference in keys - to differentiate single-day vs. multiple-day returns
#  in the EXTRACT_DATA() function

print(f"Single day return (keys): {agro_loc_values.get_data().keys()}")
print(f"Multiple day return (keys): {agro_loc_values.get_data(end_day='04-24').keys()}")

In [ ]:
# Need to treat single day vs multiple day returns differently
# Don't drop columns during this phase, bc of differences in single vs multi
# Create options/parameters for 'daily' vs 'total' for accumulations in the api_to_gdf(),
#  similar to the 'main' vs. 'soil'. This will allow to extract the total accumlated (forecast) values
#  in one dataframe vs. the daily accumulations (forecast) in another dataframe.
# It would be useful/necessary to have a start_date and end_date into the total accumulations data;
#  can these be taken from the parameters? How to incorporate this data into the DF?
#  Make list of dates, take first list[0] and last list[-1], and make new columns for this

#for entry in multi_day.get('dailyValues'):
    #print(entry.get('date'))
    
dates = [entry.get('date') for entry in multi_day.get('dailyValues')]

dates[0]
dates[-1]
dates

In [ ]:
agro_loc_values = AgronomicsLocationValues(
    api_key, api_secret, latitude=40.313250, longitude=-105.648222)

In [ ]:
# Function testing - DAILY VALUES
agronomics_total_accumulation = None
agronomics_daily_accumulation_iterator = None
agronomics = None
#agronomics = agro_loc_values.get_data() # Single day
# There can be a mismatch between the total accumulations 
#  and the sume of daily accumulations IF you specify more
#  than 10 days. The get_data function will return 10 day limit
#  for the daily accumulations, but the total accumulations will
#  go out to 15 days if specified for more than 10.
agronomics = agro_loc_values.get_data(end_day='04-23')  # Multi-day

latitude = agronomics.get('location').get('latitude')
longitude = agronomics.get('location').get('longitude')

# Check if more than one day
if agronomics.get('dailyValues'):

    # Do these with a separate call, just like in Soil accumulation='daily'
    #  accumulation='total'

    # Daily forecasted accumulations for each day
    agronomics_daily_accumulation = json_normalize(
        agronomics.get('dailyValues'))
    agronomics_daily_accumulation['latitude'] = latitude
    agronomics_daily_accumulation['longitude'] = longitude
    agronomics_daily_accumulation.set_index(['date'], inplace=True)

    # Add lat/lon
    # Shorten depth values to numerics (will be used in MultiIndex)
#     forecast_soil_df['depth'] = forecast_soil_df['depth'].apply(
#         lambda x: x[0:-15])

    # Total forecasted accumulations through all days
    agronomics_total_accumulation = json_normalize(
        agronomics.get('accumulations'))
    dates = [entry.get('date') for entry in agronomics.get('dailyValues')]
    agronomics_total_accumulation['date_range'] = f"{dates[0]}/{dates[-1]}"
    agronomics_total_accumulation['start_day'] = dates[0]
    agronomics_total_accumulation['end_day'] = dates[-1]
    agronomics_total_accumulation['latitude'] = latitude
    agronomics_total_accumulation['longitude'] = longitude
    agronomics_total_accumulation.set_index(['date_range'], inplace=True)
    
    

# Single day
else:
    agronomics_iterator = json_normalize(agronomics)
    agronomics_iterator['latitude'] = latitude
    agronomics_iterator['longitude'] = longitude
    agronomics_iterator.set_index(['date'], inplace=True)

# agronomics_iterator
#agronomics_daily_accumulation
#agronomics_total_accumulation

In [ ]:
# Function testing - DAILY VALUES
agronomics = agro_loc_values.get_data()
agronomics = agro_loc_values.get_data(end_day='04-24')

# Check if more than one day
if agronomics.get('dailyValues'):
    agronomics_iterator = json_normalize(agronomics.get('dailyValues'))

# Single day
else:
    agronomics_iterator = json_normalize(agronomics)

        
agronomics_iterator
        

## Level 4: Sub-sub-sub-class - AgronomicsLocationNorms

In [ ]:
API URL: https://api.awhere.com/v2/agronomics/locations/{latitude},{longitude}/agronomicnorms


In [ ]:
class AgronomicsLocationNorms(AgronomicsLocation):

    # Class variables for clean_data() function
    
    # https://developer.awhere.com/api/reference/agronomics/norms/geolocation
    
    """The average ratio of Precipitation to Potential Evapotranspiration 
    over the years specified. When this value is above 1, then more rain fell 
    than the amount of likely water loss; if it's below 1, then more water was
    likely lost than fell as rain. P/PET is most useful when calculated for a 
    range of days, as it is for this property, than for individual days."""
    
    # Single day   
    day_coord_cols = ['location.latitude', 'location.longitude']

    day_drop_cols = ['pet.units', '_links.self.href']

    day_rename_map = {
        "gdd.average": "gdd_average_total_cels",
        "gdd.stdDev": "gdd_average_total_std_dev_cels",
        "pet.average": "pet_average_total_mm",
        "pet.stdDev": "pet_average_total_std_dev_mm",
        "ppet.average": "ppet_average",
        "ppet.stdDev": "ppet_average_std_dev"
    }
     
    # Multi-day, total accumulation
    total_coord_cols = ['latitude', 'longitude']

    total_drop_cols = ['precipitation.units', 'pet.units']

    total_rename_map = {
        "gdd.average": "norms_gdd_average_total_cels",
        "gdd.stdDev": "norms_gdd_average_total_std_dev_cels",
        "precipitation.average": "norms_precip_average_total_mm",
        "precipitation.stdDev": "norms_precip_average_total_std_dev_mm",
        "pet.average": "norms_pet_average_total_mm",
        "pet.stdDev": "norms_pet_average_total_std_dev",
        "ppet.average": "norms_ppet_average_total",
        "ppet.stdDev": "norms_ppet_average_total_std_dev"
    }

    # Multi-day, daily accumulation
    daily_coord_cols = ['latitude', 'longitude']

    daily_drop_cols = ['pet.units', 'accumulatedPrecipitation.units',
                       'accumulatedPet.units', '_links.self.href']

    daily_rename_map = {       
        "gdd.average": "norms_gdd_average_daily_cels",
        "gdd.stdDev": "norms_gdd_average_daily_std_dev_cels",
        "pet.average": "norms_pet_average_daily_mm",
        "pet.stdDev": "norms_pet_average_daily_std_dev_mm",
        "ppet.average": "norms_ppet_average_daily",
        "ppet.stdDev": "norms_ppet_average_daily_std_dev",
        "accumulatedGdd.average": "norms_gdd_average_rolling_accum",
        "accumulatedGdd.stdDev": "norms_gdd_average_rolling_accum_std_dev",
        "accumulatedPrecipitation.average": "norms_precip_average_rolling_accum_mm",
        "accumulatedPrecipitation.stdDev": "norms_precip_average_rolling_accum_std_dev_mm",
        "accumulatedPet.average": "norms_pet_average_rolling_accum_mm",
        "accumulatedPet.stdDev": "norms_pet_average_rolling_accum_std_dev_mm", 
        "accumulatedPpet.average": "norms_ppet_average_rolling_accum",
        "accumulatedPpet.stdDev": "norms_ppet_average_rolling_accum_std"
    }

    # Define lat/lon when intitializing class; no need to repeat for lat/lon
    #  in get_data() because it is already programmed into api_url
    def __init__(self, api_key, api_secret, latitude, longitude, base_64_encoded_secret_key=None,
                 auth_token=None, api_url=None):

        super(AgronomicsLocationNorms, self).__init__(
            api_key, api_secret, base_64_encoded_secret_key, auth_token, api_url)

        self.latitude = latitude
        self.longitude = longitude
        self.api_url = f"{self.api_url}/{self.latitude},{self.longitude}/agronomicnorms"

    def get_data(self, start_day='01-01', end_day=None, offset=0):
        """Returns aWhere Historic Agronomic Norms.
        """

        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}"
        }

        # Perform the HTTP request to obtain the norms for the Field
        # Define URL variants
        url_single_day = f"{self.api_url}/{start_day}"
        url_multiple_days = f"{self.api_url}/{start_day},{end_day}?limit=10&offset={offset}"

        # Get single day norms or date range
        response = requests.get(url_multiple_days, headers=auth_headers) if end_day else requests.get(
            url_single_day, headers=auth_headers)

        # Return the norms
        return response.json()

    @staticmethod
    def extract_data(agronomic_norms):
        """Extracts data from the aWhere agronomic norms
        data in JSON format.
        """
        # Extract lat/lon
        latitude = agronomic_norms.get('location').get('latitude')
        longitude = agronomic_norms.get('location').get('longitude')

        # Check if more than one day
        if agronomic_norms.get('dailyNorms'):

            # DAILY ACCUMULATION NORMS
            # Get daily accumulation norms
            daily_norms = json_normalize(
                agronomic_norms.get('dailyNorms'))

            # Add lat/lon and set date as index
            daily_norms['latitude'] = latitude
            daily_norms['longitude'] = longitude
            daily_norms.set_index(['day'], inplace=True)

            # TOTAL ACCUMULATION NORMS
            # Get average accumulations through all days
            total_norms = json_normalize(
                agronomic_norms.get('averageAccumulations'))

            # Get list of dates, add start/end dates, set date range as index
            dates = [entry.get('day')
                     for entry in agronomic_norms.get('dailyNorms')]
            total_norms['date_range'] = f"{dates[0]}/{dates[-1]}"
            total_norms['start_day'] = dates[0]
            total_norms['end_day'] = dates[-1]
            total_norms.set_index(['date_range'], inplace=True)

            # Add lat/lon
            total_norms['latitude'] = latitude
            total_norms['longitude'] = longitude

            # Put dataframes in tuple (total norms, daily norms)
            agronomics_df = (total_norms, daily_norms)

        # Single day
        else:
            agronomics_df = json_normalize(agronomic_norms)
            # agronomics_df['latitude'] = latitude
            # agronomics_df['longitude'] = longitude
            agronomics_df.set_index(['day'], inplace=True)

        return agronomics_df

    @classmethod
    def api_to_gdf(cls, api_object, value_type='single_day', kwargs=None):
        """
        value_type can be 'single_day' or 'multi_day'.

        kwargs is a dictionary that provides values beyond the default;
        unpack dictionary if it exists

        kwargs are the parameters to get_data() method

        kwargs={'start_day': '03-04', 'end_day': '03-07', 'offset': 2}
        """
        api_data_json = api_object.get_data(
            **kwargs) if kwargs else api_object.get_data()

        if value_type.lower() == 'single_day':
            api_data_df = cls.extract_data(api_data_json)

            api_data_gdf = cls.clean_data(
                api_data_df,
                cls.day_coord_cols,
                cls.day_drop_cols,
                cls.day_rename_map
            )

        elif value_type.lower() == 'multi_day':
            api_data_df_total, api_data_df_daily = cls.extract_data(
                api_data_json)

            api_data_gdf_total = cls.clean_data(
                api_data_df_total,
                cls.total_coord_cols,
                cls.total_drop_cols,
                cls.total_rename_map
            )

            api_data_gdf_daily = cls.clean_data(
                api_data_df_daily,
                cls.daily_coord_cols,
                cls.daily_drop_cols,
                cls.daily_rename_map
            )

            api_data_gdf = (api_data_gdf_total, api_data_gdf_daily)

        else:
            raise ValueError("Invalid value type. Please choose 'single_day' or 'multi_day'.")

        return api_data_gdf

In [ ]:
# Define AgronomicsLocationNorms object - single day
agro = AgronomicsLocationNorms(
    api_key, api_secret, latitude=40.313250, longitude=-105.648222)

In [ ]:
day = AgronomicsLocationNorms.api_to_gdf(
    agro, value_type='single_day', kwargs={'start_day': '05-05'})

In [ ]:
day

In [ ]:
total_accum, daily_accum = AgronomicsLocationNorms.api_to_gdf(
    agro, value_type='multi_day', kwargs={'start_day': '05-05', 'end_day': '05-14'})

In [ ]:
total_accum

In [ ]:
daily_accum

In [ ]:
agro_norms = AgronomicsLocationNorms(
    api_key, api_secret, latitude=40.313250, longitude=-105.648222)

In [ ]:
agro_norms.api_url

In [ ]:
# Single day agronomic norms
agro_norms.get_data(start_day='05-05')#.get('location')

In [ ]:
agro_norms.get_data(start_day='05-05', end_day='05-14')

In [ ]:
# Multi day agronomic norms - average accums in this period
json_normalize(agro_norms.get_data(start_day='05-05', end_day='05-14').get('averageAccumulations'))

In [ ]:
# Multi day agronomic norms - daily accums in this period
json_normalize(agro_norms.get_data(start_day='05-05', end_day='05-14').get('dailyNorms')).tail()

In [ ]:
agro_norms = AgronomicsLocationNorms(
    api_key, api_secret, latitude=40.313250, longitude=-105.648222)

In [ ]:
agronomic_norms = agro_norms.get_data(start_day='05-05')
day = AgronomicsLocationNorms.extract_data(agronomic_norms)
day

In [ ]:
agronomic_norms = agro_norms.get_data(start_day='05-05', end_day='05-14')

In [ ]:
total, daily = AgronomicsLocationNorms.extract_data(agronomic_norms)

In [ ]:
total

In [ ]:
daily

In [ ]:

# Extract lat/lon
latitude = agronomic_norms.get('location').get('latitude')
longitude = agronomic_norms.get('location').get('longitude')

# Check if more than one day
if agronomic_norms.get('dailyNorms'):

    # DAILY ACCUMULATION NORMS
    # Get daily accumulation norms
    daily_norms = json_normalize(
        agronomic_norms.get('dailyNorms'))

    # Add lat/lon and set date as index
    daily_norms['latitude'] = latitude
    daily_norms['longitude'] = longitude
    daily_norms.set_index(['day'], inplace=True)

    # TOTAL ACCUMULATION NORMS
    # Get average accumulations through all days
    total_norms = json_normalize(
        agronomic_norms.get('averageAccumulations'))

    # Get list of dates, add start/end dates, set date range as index
    dates = [entry.get('day')
             for entry in agronomic_norms.get('dailyNorms')]
    total_norms['date_range'] = f"{dates[0]}/{dates[-1]}"
    total_norms['start_day'] = dates[0]
    total_norms['end_day'] = dates[-1]
    total_norms.set_index(['date_range'], inplace=True)

    # Add lat/lon
    total_norms['latitude'] = latitude
    total_norms['longitude'] = longitude

    # Put dataframes in tuple (total norms, daily norms)
    agronomics_df = (total_norms, daily_norms)

# Single day
else:
    agronomics_df = json_normalize(agronomic_norms)
    # agronomics_df['latitude'] = latitude
    # agronomics_df['longitude'] = longitude
    agronomics_df.set_index(['day'], inplace=True)

return agronomics_df

In [ ]:
total, daily = agronomics_df

In [ ]:
total

In [ ]:
daily

In [ ]:
class AgronomicsCrops(Agronomics):

    def __init__(self, api_key, api_secret, base_64_encoded_secret_key=None,
                 auth_token=None, api_url=None):

        super(AgronomicsCrops, self).__init__(
            api_key, api_secret, base_64_encoded_secret_key, auth_token, api_url)

        self.api_url = f"{self.api_url}/crops"

    # field_id=None, crop_name=None, limit=10, offset=0
    def get(self, crop_id=None, limit=10, offset=0):
        """Retrieve a list of available crops.
        """
        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}"
        }

        # Get API response, single crop or page of crops
        response = requests.get(f"{self.api_url}/{crop_id}", headers=auth_headers) if crop_id else requests.get(
            f"{self.api_url}?limit={limit}&offset={offset}", headers=auth_headers)

        # Convert API response to JSON format
        response_json = response.json()

        # Conveer to dataframe
        response_df = json_normalize(response_json) if crop_id else json_normalize(
            response_json.get('crops'))

        # Drop unnecessary columns
        response_df.drop(columns=[
            '_links.self.href', '_links.curies', '_links.awhere:plantings.href'
        ], inplace=True)

        # Reset index
        response_df.reset_index(drop=True, inplace=True)

        # Define new column names
        crop_rename = {
            'id': 'crop_id',
            'name': 'crop_name',
            'type': 'crop_type',
            'variety': 'crop_variety',
            'isDefaultForCrop': 'default_crop'
        }

        # Rename columns
        response_df.rename(columns=crop_rename, inplace=True)

        return response_df

    def get_full(self, limit=10, offset=0, max_pages=3):
        """Retrieves the full list of available crops,
        based on limit, offset, and max pages.
        """
        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}"
        }

        # Define list to store page dataframes
        response_df_list = []

        # Loop through all pages
        while offset < limit * max_pages:

            # Get API response; convert response to dataframe; append to dataframe list
            response = requests.get(
                f"{self.api_url}?limit={limit}&offset={offset}", headers=auth_headers)
            response_json = response.json()
            response_df_loop = json_normalize(response_json.get('crops'))
            response_df_list.append(response_df_loop)
            offset += 10

        # Merge all dataframes into a single dataframe
        response_df = pd.concat(response_df_list, axis=0)

        # Drop unnecessary dataframe columns
        response_df.drop(columns=[
            '_links.self.href', '_links.curies', '_links.awhere:plantings.href'
        ], inplace=True)

        # Reset dataframe index
        response_df.reset_index(drop=True, inplace=True)

        # Define new column name mapping
        crop_rename = {
            'id': 'crop_id',
            'name': 'crop_name',
            'type': 'crop_type',
            'variety': 'crop_variety',
            'isDefaultForCrop': 'default_crop'
        }

        # Rename dataframe columns
        response_df.rename(columns=crop_rename, inplace=True)

        return response_df

In [ ]:
a = AgronomicsCrops(api_key, api_secret)

In [ ]:
a.api_url

In [ ]:
a.get()

In [ ]:
a.get(crop_id='canola-b-rapa')

In [ ]:
l = a.get_full()

In [ ]:
a.get_full()

In [ ]:
a.get().get('next', 'Wrong level')

In [ ]:
json_normalize(a.get())

In [ ]:
a.get().keys()

In [ ]:
json_normalize(a.get().get('crops'))

In [ ]:
class AgronomicsCrop(AgronomicsCrops):

    def __init__(self, api_key, api_secret, crop_id, base_64_encoded_secret_key=None,
                 auth_token=None, api_url=None):

        super(AgronomicsCrop, self).__init__(
            api_key, api_secret, base_64_encoded_secret_key, auth_token, api_url)

        self.api_url = f"{self.api_url}/{crop_id}"
        
    def get(self):
        """Retrieves the information for the defined crop.
        """
        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}"
        }

        # Get API response, single crop
        response = requests.get(f"{self.api_url}", headers=auth_headers)
    
        # Convert API response to JSON format
        response_json = response.json()

        # Conveer to dataframe
        response_df = json_normalize(response_json)

        # Drop unnecessary columns
        response_df.drop(columns=[
            '_links.self.href', '_links.curies', '_links.awhere:plantings.href'
        ], inplace=True)

        # Reset index
        response_df.reset_index(drop=True, inplace=True)

        # Define new column names
        crop_rename = {
            'id': 'crop_id',
            'name': 'crop_name',
            'type': 'crop_type',
            'variety': 'crop_variety',
            'isDefaultForCrop': 'default_crop'
        }

        # Rename columns
        response_df.rename(columns=crop_rename, inplace=True)

        return response_df

In [ ]:
b = AgronomicsCrop(api_key, api_secret, 'corn-2300-gdd')

In [ ]:
b.api_url

In [ ]:
b.get()

In [ ]:
class AgronomicsFieldPlantings(AgronomicsField):

    # Define field_id intitializing class
    def __init__(self, api_key, api_secret, field_id, base_64_encoded_secret_key=None,
                 auth_token=None, api_url=None):

        super(AgronomicsFieldPlantings, self).__init__(
            api_key, api_secret, base_64_encoded_secret_key, auth_token, api_url)

        self.field_id = field_id
        self.api_url = f"{self.api_url}/{self.field_id}/plantings"

    def get(self, planting_id=None, limit=10, offset=0):
        """Returns aWhere plantings associated with a specified field.

        planting_id can either be an actual id or 'current' for the most
        current planting. 'None' will result in all plantings

        GET /v2/agronomics/fields/{fieldId}/plantings
        GET /v2/agronomics/fields/{fieldId}/plantings/{plantingId}
        GET /v2/agronomics/fields/{fieldId}/plantings/current        
        """

        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}"
            # "Content-Type": 'application/json'
        }

        # Get API response
        response = requests.get(f"{self.api_url}/{planting_id}", headers=auth_headers) if planting_id else requests.get(
            f"{self.api_url}?limit={limit}&offset={offset}", headers=auth_headers)
        
        # Convert API response to JSON format
        response_json = response.json()

        # Convert to dataframe
        response_df = json_normalize(response_json) if planting_id else json_normalize(
            response_json.get('plantings'))

        drop_cols = [
            '_links.self.href', '_links.curies', 
            '_links.awhere:crop.href', '_links.awhere:field.href'
        ]
        
        # Drop unnecessary columns
        response_df.drop(
            columns=drop_cols, inplace=True)
            
        # Define new column names
        planting_rename = {
            'id': 'planting_id',
            'crop': 'crop_id',
            'field': 'field_id',
            'plantingDate': 'planting_date',
            'harvestDate': 'harvest_date_actual',
            # What is 'recommendation' field? What output goes here, and where does it come from?
            'yield.amount': 'yield_amount_actual', #{response_json.get("yield").get("units").lower()}',
            'yield.units': 'yield_amount_actual_units',
            'projections.yield.amount': 'yield_amount_projected', #{response_json.get("projections").get("yield").get("units").lower()}',
            'projections.yield.units': 'yield_amount_projected_units',
            'projections.harvestDate': 'harvest_date_projected' 
        }

        # Rename
        response_df.rename(columns=planting_rename, inplace=True)

        # Set index
        response_df.set_index('planting_id', inplace=True)
        
        return response_df

    def create(self, crop, planting_date, projected_yield_amount=None, projected_yield_units=None,
               projected_harvest_date=None, yield_amount=None, yield_units=None, harvest_date=None):
        """Creates a planting in the field.
        """
        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {str(self.auth_token)}",
            "Content-Type": 'application/json'
        }

        # Define request body
        field_body = {
            "crop": crop,
            "plantingDate": planting_date,
            "projections": {
                "yield": {
                    "amount": projected_yield_amount,
                    "units": projected_yield_units,
                },
                "harvestDate": projected_harvest_date
            },
            "yield": {
                "amount": yield_amount,
                "units": yield_units,
            },
            "harvestDate": harvest_date
        }
        
        # Creat planting
        response = requests.post(
            self.api_url, headers=auth_headers, json=field_body)

        return response.json()

    def update(self, planting_id='current', update_type='replace', kwargs=None):
        """Update a planting. update_type can be 'replace' or 'update'
        
        kwargs is a dict with all the update values
        
        PUT /v2/agronomics/fields/{fieldId}/plantings/{plantingId}
        PUT /v2/agronomics/fields/{fieldId}/plantings/current
        
        update_kwargs = {
            "crop": 'wheat-hardred',
            "planting_date": '2019-05-20', 
            "projected_yield_amount": 90, 
            "projected_yield_units": 'small boxes',
            "projected_harvest_date": "2019-08-10", 
            "yield_amount": 100, 
            "yield_units": "medium boxes", 
            "harvest_date": '2019-08-31'
        }
    
        PATCH /v2/agronomics/fields/{fieldId}/plantings/{plantingId}
        PATCH /v2/agronomics/fields/{fieldId}/plantings/current
        
        Use dict comprehension for updates
        """
        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {str(self.auth_token)}",
            "Content-Type": 'application/json'
        }   
    
        # Full replace
        if update_type.lower() == 'replace':
        # Define request body
            field_body = {
                "crop": kwargs.get('crop'),
                "plantingDate": kwargs.get('planting_date'),
                "projections": {
                    "yield": {
                        "amount": kwargs.get('projected_yield_amount'),
                        "units": kwargs.get('projected_yield_units'),
                    },
                    "harvestDate": kwargs.get('projected_harvest_date')
                },
                "yield": {
                    "amount": kwargs.get('yield_amount'),
                    "units": kwargs.get('yield_units'),
                },
                "harvestDate": kwargs.get('harvest_date')
            }
        
            # Update planting
            response = requests.put(
                f"{self.api_url}/{planting_id}", headers=auth_headers, json=field_body)
            
        elif update_type.lower() == 'update':
            
            # Define field body
            field_body = [{"op": "replace", "path": f"/{key}", "value": f"{value}"}
                         for key, value in kwargs.items()]
            
            # Perform the HTTP request to update field information
            response = requests.patch(
                f"{self.api_url}/{planting_id}", headers=auth_headers, json=field_body)
            
        else:
            raise ValueError("Invalid update type. Please choose 'replace' or 'update'.")

        return response.json()
        
    def delete(self, planting_id='current'):
        """Deletes a planting, based on planting id or
        the most recent planting (based on id)
        """
         # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}",
            #"Content-Type": 'application/json'
        }

        # Perform the POST request to Delete the Field
        response = requests.delete(
            f"{self.api_url}/{planting_id}", headers=auth_headers)

        message = f"Deleted planting: {planting_id}" if response.status_code == 204 else f"Could not delete planting."

        return print(message)

In [ ]:
#  # Define request body
#             field_body = [{
#                 "op": "replace",
#                 "path": f"/{key}",
#                 "value": f"{value}"
#             } for key:value in kwargs]
           
dict_list = []
for key, value in update_kwargs.items():
    new_dict = {
        "op": "replace",
        "path": f"/{key}",
        "value": f"{value}"
    }
    dict_list.append(new_dict)

In [ ]:
dict_list

In [ ]:
dict_list_comp = [{"op": "replace", "path": f"/{key}", "value": f"{value}"}
                  for key, value in update_kwargs.items()]

In [ ]:
dict_list_comp

In [ ]:
plantings_object = AgronomicsFieldPlantings(
    api_key, api_secret, 'CO-RMNP-Bear-Lake')

In [ ]:
update_kwargs = {
    "crop": 'wheat-hardred',
    "planting_date": '2019-05-20', 
    "projected_yield_amount": 90, 
    "projected_yield_units": 'small boxes',
    "projected_harvest_date": "2019-08-10", 
    "yield_amount": 100, 
    "yield_units": "medium boxes", 
    "harvest_date": '2019-08-31'
}
    

In [ ]:
plantings_object.update(update_type='update', kwargs=update_kwargs)

In [ ]:
plantings_object.api_url

In [ ]:
planting = plantings_object.create(
    crop='sorghum-long-season', planting_date='2019-05-05', 
    projected_yield_amount=80, projected_yield_units='boxes',
    yield_amount=75, yield_units='boxes')

In [ ]:
plantings_object.get()

In [ ]:
plantings_object.delete(464536787)

In [ ]:
df, json = plantings_object.get(planting_id='current')

In [ ]:
df

In [ ]:
json.get('projections').get('yield').get('units').lower()

In [ ]:
str(json.get('projections').get('yield').get('units')).lower()

In [ ]:
str(json.get('yield').get('units')).lower()

In [ ]:
plantings_object.get(planting_id='current') # most recent (based on id)

In [ ]:
plantings_object.get(planting_id=464533) 

In [ ]:
plantings_object.get()

In [ ]:
plantings_object.field_id

In [ ]:
json_normalize(plantings_object.get().get('plantings'))

In [ ]:
planting = plantings_object.create(crop='wheat-hardred', planting_date='2019-05-05')

In [ ]:
json_normalize(planting)

In [ ]:
# class AgronomicsFieldPlanting(AgronomicsField):

#     # Define field_id intitializing class
#     def __init__(self, api_key, api_secret, field_id, planting_id, base_64_encoded_secret_key=None,
#                  auth_token=None, api_url=None):
        
#         """planting_id can either be 'current' or the specific planting_id for a field
#         """
#         super(AgronomicsFieldPlanting, self).__init__(
#             api_key, api_secret, base_64_encoded_secret_key, auth_token, field_id)

#         self.field_id = field_id
#         self.api_url = f"{self.api_url}"#"/{self.field_id}/plantings/{planting_id}"
        
        
#     def get(self):
#         """Retrieve info about the planting.
#         """
#         # Setup the HTTP request headers
#         auth_headers = {
#             "Authorization": f"Bearer {self.auth_token}"
#             # "Content-Type": 'application/json'
#         }

#         # Get API response
#         response = requests.get(f"{self.api_url}", headers=auth_headers) 
        
#         # Convert API response to JSON format
#         response_json = response.json()

# #         # Convert to dataframe
# #         response_df = json_normalize(response_json)

# #         drop_cols = [
# #             '_links.self.href', '_links.curies', 
# #             '_links.awhere:crop.href', '_links.awhere:field.href'
# #         ]
        
# #         # Drop unnecessary columns
# # #         response_df.drop(
# # #             columns=drop_cols, inplace=True)
            
# #         # Define new column names
# #         planting_rename = {
# #             'id': 'planting_id',
# #             'crop': 'crop_id',
# #             'field': 'field_id',
# #             'plantingDate': 'planting_date',
# #             'harvestDate': 'harvest_date_actual',
# #             # What is 'recommendation' field? What output goes here, and where does it come from?
# #             'yield.amount': 'yield_amount_actual', #{response_json.get("yield").get("units").lower()}',
# #             'yield.units': 'yield_amount_actual_units',
# #             'projections.yield.amount': 'yield_amount_projected', #{response_json.get("projections").get("yield").get("units").lower()}',
# #             'projections.yield.units': 'yield_amount_projected_units',
# #             'projections.harvestDate': 'harvest_date_projected' 
# #         }

# #         # Rename
# #         response_df.rename(columns=planting_rename, inplace=True)

# #         # Set index
# #         response_df.set_index('planting_id', inplace=True)
        
#         return response_json
        

In [ ]:
c = AgronomicsFieldPlanting(
    api_key, api_secret, field_id='CO-RMNP-Bear-Lake', planting_id='current')

In [ ]:
c.api_url

In [ ]:
planting_object = AgronomicsFieldPlanting(
    api_key, api_secret, field_id='CO-RMNP-Bear-Lake', planting_id=464536)

In [ ]:
planting_object.api_url

In [ ]:
planting_object.get()

In [ ]:
class Planting(AgronomicsFieldPlantings):

    # Define field_id intitializing class
    def __init__(self, api_key, api_secret, field_id, base_64_encoded_secret_key=None,
                 auth_token=None, api_url=None):

        super(Planting, self).__init__(
            api_key, api_secret, base_64_encoded_secret_key, auth_token, api_url)

        self.field_id = field_id
        #self.api_url = f"{self.api_url}/{self.field_id}/plantings"

In [ ]:
x = Planting(api_key, api_secret, field_id='RMNP-CO-Bear-Lake')

In [ ]:
x.api_url

In [ ]:
class AgronomicsModels(Agronomics):

    def __init__(self, api_key, api_secret, base_64_encoded_secret_key=None,
                 auth_token=None, api_url=None):

        super(AgronomicsModels, self).__init__(
            api_key, api_secret, base_64_encoded_secret_key, auth_token, api_url)

        self.api_url = f"{self.api_url}/models"

    def get(self, model_id=None, limit=10, offset=0):
        """Retrieve a list of available models.
        """
        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}"
        }

        # Get API response, single crop or page of crops
        response = requests.get(f"{self.api_url}/{model_id}", headers=auth_headers) if model_id else requests.get(
            f"{self.api_url}?limit={limit}&offset={offset}", headers=auth_headers)

        # Convert API response to JSON format
        response_json = response.json()

        # Convert to dataframe
        response_df = json_normalize(response_json) if model_id else json_normalize(
            response_json.get('models'))

        # Drop unnecessary columns
        response_df.drop(columns=[
            '_links.self.href', '_links.curies', 
            '_links.awhere:crop', '_links.awhere:modelDetails.href'
        ], inplace=True)

        # Define new column names
        model_rename = {
            'id': 'model_id',
            'name': 'model_name',
            'description': 'model_description',
            'type': 'model_type',
            'source.name': 'model_source',
            'source.link': 'model_link'
        }

        # Rename columns
        response_df.rename(columns=model_rename, inplace=True)

        # Set index
        response_df.set_index('model_id', inplace=True)

        return response_df

    def get_full(self, limit=10, offset=0, max_pages=3):
        """Retrieves the full list of available models,
        based on limit, offset, and max pages.
        """
        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}"
        }

        # Define list to store page dataframes
        response_df_list = []

        # Loop through all pages
        while offset < limit * max_pages:

            # Get API response; convert response to dataframe; append to dataframe list
            response = requests.get(
                f"{self.api_url}?limit={limit}&offset={offset}", headers=auth_headers)
            response_json = response.json()
            response_df_loop = json_normalize(response_json.get('models'))
            response_df_list.append(response_df_loop)
            offset += 10

        # Merge all dataframes into a single dataframe
        response_df = pd.concat(response_df_list, axis=0)
        
          # Drop unnecessary columns
        response_df.drop(columns=[
            '_links.self.href', '_links.curies', 
            '_links.awhere:crop', '_links.awhere:modelDetails.href'
        ], inplace=True)

        # Define new column names
        model_rename = {
            'id': 'model_id',
            'name': 'model_name',
            'description': 'model_description',
            'type': 'model_type',
            'source.name': 'model_source',
            'source.link': 'model_link'
        }

        # Rename columns
        response_df.rename(columns=model_rename, inplace=True)

        # Set index
        response_df.set_index('model_id', inplace=True)

        return response_df
    
    
    def get_details(self, model_id='BarleyGenericMSU'):
        """Retrieve model details
        """
         # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}"
        }

        # Get API response, single crop or page of crops
        response = requests.get(f"{self.api_url}/{model_id}/details", headers=auth_headers)

        # Convert API response to JSON format
        response_json = response.json()
        
        # Model base information
        base_info_df = json_normalize(response_json)
        base_info_df.drop(
            columns=[
                'gddUnits', 'stages', '_links.self.href', 
                '_links.curies', '_links.awhere:model.href'
            ], inplace=True)
        base_info_df['model_id'] = model_id
        base_info_df.set_index('model_id', inplace=True)
        base_info_df.rename(columns={
            'biofix': 'biofix_days',
            'gddMethod': 'gdd_method',
            'gddBaseTemp': 'gdd_base_temp_cels',
            'gddMaxBoundary': 'gdd_max_boundary_cels',
            'gddMinBoundary': 'gdd_min_boundary_cels'
        }, inplace=True)
        
        # Model stage information
        stage_info_df = json_normalize(response_json.get('stages'))
        stage_info_df.drop(columns=['gddUnits'], inplace=True)
        stage_info_df['model_id'] = model_id
        stage_info_df.rename(columns={
            'id': 'stage_id',
            'stage': 'stage_name',
            'description': 'stage_description',
            'gddThreshold': 'gdd_threshold_cels',
        }, inplace=True)
        stage_info_df.set_index(['model_id', 'stage_id'], inplace=True)
        
        # Return base info and stage info dataframes
        return base_info_df, stage_info_df
    
#     def get_all_details(self):
#         """Get dataframes with details on all
#         available models.
#         """
#         # Lists to store dataframes
#         base_list = []
#         stage_list = []

#         for model in list(self.get_full().index):
#             base, stage = self.get_details(model_id=model)
#             base_list.append(base)
#             stage_list.append(stage)

#         base_df_all = pd.concat(base_list, axis=0)
#         stage_df_all = pd.concat(stage_list, axis=0)
        
#         return base_df_all, stage_df_all
    
    @classmethod
    def get_all_details(cls, api_object):
        """Get dataframes with details on all
        available models.
        """
        # Lists to store dataframes
        base_list = []
        stage_list = []

        for model in list(api_object.get_full().index):
            base, stage = api_object.get_details(model_id=model)
            base_list.append(base)
            stage_list.append(stage)

        base_df_all = pd.concat(base_list, axis=0)
        stage_df_all = pd.concat(stage_list, axis=0)
        
        return base_df_all, stage_df_all

In [ ]:
model_object = AgronomicsModels(api_key, api_secret)

In [ ]:
c, t = AgronomicsModels.get_all_details(model_object)

In [ ]:
t

In [ ]:
b, s = model_object.get_all_details()

In [ ]:
s

In [ ]:
b

In [ ]:
combined = model_object.get_details()

In [ ]:
combined

In [ ]:
base, stage = model_object.get_details()

In [ ]:
base

In [ ]:
stage

In [ ]:
json_normalize(model_object.get_details())

In [ ]:
json_normalize(model_object.get_details().get('stages'))

In [ ]:
base_df_all

In [ ]:
stage_df_all

In [ ]:
model_object.get(model_id='BarleyGenericNDAWN')

In [ ]:
model_object.get()

In [ ]:
model_object.api_url

In [ ]:
json_normalize(model_object.get(model_id='BarleyGenericNDAWN'))

In [ ]:
json_normalize(model_object.get().get('models'))

In [ ]:
class AgronomicsPlantings(Agronomics):

    def __init__(self, api_key, api_secret, base_64_encoded_secret_key=None,
                 auth_token=None, api_url=None):

        super(AgronomicsPlantings, self).__init__(api_key, api_secret,
                                                  base_64_encoded_secret_key, auth_token)

        self.api_url = f'{self.api_url}/plantings'

    def get(self, planting_id=None, limit=10, offset=0):
        """Returns aWhere plantings associated with your account.

        planting_id can either be an actual id or 'current' for the most
        current planting. 'None' will result in all planting.      
        """
        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}"
            # "Content-Type": 'application/json'
        }

        # Get API response
        response = requests.get(f"{self.api_url}/{planting_id}", headers=auth_headers) if planting_id else requests.get(
            f"{self.api_url}?limit={limit}&offset={offset}", headers=auth_headers)

        # Convert API response to JSON format
        response_json = response.json()

        # Convert to dataframe
        response_df = json_normalize(response_json.get('plantings')) if response_json.get(
            'plantings') else json_normalize(response_json)

        drop_cols = [
            '_links.self.href', '_links.curies',
            '_links.awhere:crop.href', '_links.awhere:field.href'
        ]

        # Drop unnecessary columns
        response_df.drop(
            columns=drop_cols, inplace=True)

        # Define new column names
        planting_rename = {
            'id': 'planting_id',
            'crop': 'crop_id',
            'field': 'field_id',
            'plantingDate': 'planting_date',
            'harvestDate': 'harvest_date_actual',
            # What is 'recommendation' field? What output goes here, and where does it come from?
            # {response_json.get("yield").get("units").lower()}',
            'yield.amount': 'yield_amount_actual',
            'yield.units': 'yield_amount_actual_units',
            # {response_json.get("projections").get("yield").get("units").lower()}',
            'projections.yield.amount': 'yield_amount_projected',
            'projections.yield.units': 'yield_amount_projected_units',
            'projections.harvestDate': 'harvest_date_projected'
        }

        # Rename
        response_df.rename(columns=planting_rename, inplace=True)

        # Set index
        response_df.set_index('planting_id', inplace=True)

        return response_df

In [ ]:
agro_plant = AgronomicsPlantings(api_key, api_secret)

In [ ]:
agro_plant.api_url

In [ ]:
agro_plant.get()

In [ ]:
agro_plant.get(planting_id=464537)

In [ ]:
agro_plant.get(planting_id='current')

In [ ]:
class AgronomicsFieldModels(AgronomicsField):

    # Define field_id intitializing class
    def __init__(self, api_key, api_secret, field_id, model_id, base_64_encoded_secret_key=None,
                 auth_token=None, api_url=None):

        super(AgronomicsFieldModels, self).__init__(
            api_key, api_secret, base_64_encoded_secret_key, auth_token, api_url)

        self.field_id = field_id
        self.model_id = model_id
        self.api_url = f"{self.api_url}/{self.field_id}/models/{model_id}/results"

    def get(self):
        """Returns aWhere model associated with a field.      
        """
        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}"
            # "Content-Type": 'application/json'
        }

        # Get API response
        response = requests.get(
            f"{self.api_url}", headers=auth_headers)

        # Convert API response to JSON format
        response_json = response.json()

        # Get stage info
        previous_stage_df = json_normalize(response_json.get('previousStages'))
        current_stage_df = json_normalize(response_json.get('currentStage'))
        next_stage_df = json_normalize(response_json.get('nextStage'))

        # Add columns
        previous_stage_df['stage_status'] = 'Previous'
        current_stage_df['stage_status'] = 'Current'
        next_stage_df['stage_status'] = 'Next'

        # Merge into one dataframe
        stages_df = pd.concat([
            previous_stage_df, current_stage_df, next_stage_df],
            sort=False, axis=0)

        # Change column names
        stages_df.rename(columns={
            'date': 'stage_start_date',
            'id': 'stage_id',
            'stage': 'stage_name',
            'description': 'stage_description',
            'gddThreshold': 'gdd_threshold_cels',
            'accumulatedGdds': 'gdd_accumulation_current_cels',
            'gddRemaining': 'gdd_remaining_next_cels'
        }, inplace=True)

        # Add base data
        stages_df['biofix_date'] = response_json.get('biofixDate')
        stages_df['planting_date'] = response_json.get('plantingDate')
        stages_df['model_id'] = response_json.get('modelId')
        stages_df['field_id'] = response_json.get('location').get('fieldId')
        stages_df['longitude'] = response_json.get('location').get('longitude')
        stages_df['latitude'] = response_json.get('location').get('latitude')

        # Set index
        stages_df.set_index(['field_id', 'stage_status'], inplace=True)

        # Prep for geodataframe conversion
        df_copy = stages_df.copy()

        # Define CRS (EPSG 4326)
        crs = {'init': 'epsg:4326'}

        # Convert to geodataframe
        stages_gdf = gpd.GeoDataFrame(
            df_copy, crs=crs, geometry=gpd.points_from_xy(
                stages_df.longitude,
                stages_df.latitude)
        )

        # Drop lat/lon
        stages_gdf.drop(columns=['longitude', 'latitude'], inplace=True)

        # Reorder columns
        stages_gdf = stages_gdf.reindex(columns=[
            'model_id', 'biofix_date', 'planting_date',
            'stage_start_date', 'stage_id', 'stage_name',
            'stage_description', 'gdd_threshold_cels',
            'gdd_accumulation_current_cels', 'gdd_remaining_next_cels',
            'geometry'
        ])

        return stages_gdf

In [ ]:
model_object = AgronomicsFieldModels(
    api_key, api_secret, field_id='CO-RMNP-Bear-Lake', model_id='SorghumLongSeasonTexasAM')

In [ ]:
model_object.get()

In [ ]:
model_object.get(model_id='SorghumLongSeasonTexasAM') # WheatHardRedMSU

In [ ]:
model_object.api_url

In [ ]:
# model_object.get(model_id='WheatHardRedMSU')

In [ ]:
model_object.get_base(model_id='SorghumLongSeasonTexasAM') # WheatHardRedMSU

In [ ]:
#     def get_base(self, model_id):
#         """Returns aWhere plantings associated with a specified field.

#         planting_id can either be an actual id or 'current' for the most
#         current planting. 'None' will result in all plantings

#         GET /v2/agronomics/fields/{fieldId}/plantings
#         GET /v2/agronomics/fields/{fieldId}/plantings/{plantingId}
#         GET /v2/agronomics/fields/{fieldId}/plantings/current        
#         """

#         # Setup the HTTP request headers
#         auth_headers = {
#             "Authorization": f"Bearer {self.auth_token}"
#             # "Content-Type": 'application/json'
#         }

#         # Get API response
#         response = requests.get(
#             f"{self.api_url}/{model_id}/results", headers=auth_headers)

#         # BASE INFORMATION
#         # Convert API response to JSON format
#         response_json = response.json()

#         # Convert to dataframe
#         response_df = json_normalize(response_json)

#         drop_cols = [
#             'previousStages', 'gddUnits', 'currentStage.accumulatedGdds',
#             'currentStage.date', 'currentStage.id', 'currentStage.stage',
#             'currentStage.description', 'currentStage.gddThreshold', 'nextStage.description',
#             'nextStage.gddThreshold', 'nextStage.gddRemaining', 'nextStage.id',
#             'nextStage.stage', '_links.self.href', '_links.curies', '_links.awhere:model.href',
#             '_links.awhere:modelDetails.href', '_links.awhere:field.href', '_links.awhere:planting.href'
#         ]

#         # Drop unnecessary columns
#         response_df.drop(
#             columns=drop_cols, inplace=True)

#         # Define new column names
#         model_rename = {
#             'biofixDate': 'biofix_date',
#             'modelId': 'model_id',
#             'plantingDate': 'planting_date',
#             'location.latitude': 'latitude',
#             'location.longitude': 'longitude',
#             'location.fieldId': 'field_id'
#         }

#         # Rename
#         response_df.rename(columns=model_rename, inplace=True)

#         # Set index
#         # ['field_id', 'model_id']
#         response_df.set_index('field_id', inplace=True)

#         # Prep for geodataframe conversion
#         df_copy = response_df.copy()

#         # Define CRS (EPSG 4326)
#         crs = {'init': 'epsg:4326'}

#         # Convert to geodataframe
#         response_gdf = gpd.GeoDataFrame(
#             df_copy, crs=crs, geometry=gpd.points_from_xy(
#                 response_df.longitude,
#                 response_df.latitude)
#         )

#         # Drop lat/lon
#         response_gdf.drop(columns=['longitude', 'latitude'], inplace=True)

#         return response_gdf

In [ ]:
# json_normalize(model_object.get(model_id='SorghumLongSeasonTexasAM'))

In [ ]:
json_normalize(model_object.get(model_id='SorghumLongSeasonTexasAM').get('previousStages'))

In [ ]:
json_normalize(model_object.get(model_id='SorghumLongSeasonTexasAM').get('currentStage'))

In [ ]:
json_normalize(model_object.get(model_id='SorghumLongSeasonTexasAM').get('nextStage'))

In [ ]:
pd.concat([
    json_normalize(model_object.get(model_id='SorghumLongSeasonTexasAM').get('previousStages')), 
    json_normalize(model_object.get(model_id='SorghumLongSeasonTexasAM').get('currentStage')),
    json_normalize(model_object.get(model_id='SorghumLongSeasonTexasAM').get('nextStage'))
], sort=False, axis=0)